# I/O: FITS and ASCII


### In this section we will:
- Learn how to use FITS files
    - Open a FITS file
    - Examine its header
    - Image data
    - Table data
    - Visualizing with `matplotlib`
- Learn how to use ASCII files
    - Open an ASCII file
    - Visualizing with `matplotlib`
    
#### For more information, see the Astropy documentation on [FITS](http://docs.astropy.org/en/stable/io/fits/) and [ASCII](http://docs.astropy.org/en/stable/io/ascii/index.html)
---


# Working with FITS files

In [ ]:
from astropy.io import fits

The *open()* function in [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html#) works with regular and compressed files.

Open a FITS file and look at the contents:

In [ ]:
f = fits.open('j94f05bgq_flt.fits.gz')
f.info()
f.close()

There is also a shortcut to look inside a FITS file before fully opening it:

In [ ]:
fits.info('j94f05bgq_flt.fits.gz')

In [ ]:
type(f)

FITS file objects, `HDUList`s,  are like lists. Individual HDUs are indexed like lists:

In [ ]:
f[1]

or using a (EXTNAME, EXTVER) tuple.

In [ ]:
f[("SCI", 1)]

## Working with headers

Headers are similar to python dictionaries.
Look at a header, modify an existing keyword and add a new card.

In [ ]:
with fits.open('j94f05bgq_flt.fits.gz', mode='update') as f:
    print("crpix1: ", f[1].header['crpix1'])
    f[1].header['crpix1'] = 12  # Reassign the keyword
    print("Updated crpix1 :", f[1].header['crpix1'])

Opening a FITS file using the Python *with* statement
ensures that the file is closed without explicitely doing so.

There are several ways to add a new card.

Note: Attempting to access a non-existing keyword raises a *KeyError*.

In [ ]:
f[1].header['observer'] = "Edwin Hubble"
    
# Assigning a tuple to a keyword splits
# the tuple into a keyword value and comment.
    
f[1].header['NOBS'] = (2, "Number of observing nights")
print("observer:", f[1].header['observer'])
print(f[1].header.comments['NOBS'], f[1].header['NOBS'])

To delete a keyword/card use the python *del* operator.

Again a *KeyError* is raised if the keyword is not in the header.

In [ ]:
del f[1].header['NOBS']
del f[1].header['observer']

Comment and history cards are added as regular keywords. In this case a new card is always created.

In [ ]:
with fits.open('j94f05bgq_flt.fits.gz', mode='update') as f:
    f[1].header["history"] = "New history card."
    f[1].header["comment"] = "This is a 47 Tuc observation with HST"
    f[1].header["comment"] = "I'd like to observe it with JWST."
    f[1].header['comment']
    print(f[1].header['comment'])

## Working with image data

In [ ]:
f = fits.open('pix.fits.gz')
f.info()

An image is a numpy array saved as the data part of an HDU.

In [ ]:
scidata = f[0].data
print(scidata.shape)
print(scidata.dtype)

*scidata* is a pointer to the data array of the HDU and if it changes, the data in the HDU changes as well.

In [ ]:
# All operations available to NDArray are applicable to the fits data array.
scidata[2:10, 3:7].mean()

`astropy.io.fits` provides convenience functions to operate on FITS files. These are good for interactive and quick lookup use. It is recommended to use the object-oriented interface for programming (creating a FITS object that you operate on).

In [ ]:
# Get the data from an HDU 
data = fits.getdata('pix.fits.gz')
print("min, max: ", data.min(), data.max())

In [ ]:
# Get a keyword value
wcsaxes = fits.getval('j94f05bgq_flt.fits.gz', keyword="WCSAXES", ext=1)
print("wcsaxes:", wcsaxes)

In [ ]:
# Set a keyword value
fits.setval("j94f05bgq_flt.fits.gz", keyword="observer", value="Edwin Hubble")

print("OBSERVER: ", fits.getval("j94f05bgq_flt.fits.gz", keyword="OBSERVER"))

In [ ]:
# Remove a keyword from a header
fits.delval("j94f05bgq_flt.fits.gz", keyword="OBSERVER")

### Visualizing with `matplotlib`

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

%matplotlib inline

In [ ]:
plt.imshow(scidata, vmin=50, vmax=200)
plt.colorbar()

For astronomical images, you often want the image scaling to be logarithmic. You can do this a number of ways, but the recommended was is to define the `norm` argument to be `matplotlib.colors.LogNorm()`.

In [ ]:
plt.imshow(scidata, vmin=50, vmax=200, norm=LogNorm())
plt.colorbar()

`matplotlib` has a number of built-in color maps that can be used to represent data. See all the options [on the `matplotlib` website](https://matplotlib.org/examples/color/colormaps_reference.html). 

In [ ]:
plt.imshow(scidata, vmin=50, vmax=200, norm=LogNorm(), cmap='gray')
plt.colorbar()

## Working with FITS tables

**Note**: The recommended method to read and write a single FITS table is using the [Unified I/O read/write interface](http://docs.astropy.org/en/stable/io/unified.html#table-io-fits):

    from astropy.table import Table
    t = Table.read('data.fits')

However, here we show an example of using [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html#) as there exists a lot of legacy code which uses it.

In [ ]:
from astropy.utils.data import download_file

In [ ]:
table_filename = download_file( 'http://data.astropy.org/tutorials/FITS-tables/chandra_events.fits', cache=True )

In [ ]:
hdu_list = fits.open(table_filename)
hdu_list.info()

In [ ]:
table_data = hdu_list[1].data
print("Column names: \n", table_data.names)

print("\nRow 1: \n", table_data[1])

print('\nColumn "time": \n', table_data.field("time"))

print("\nNumber of rows: \n", len(table_data))

### Visualizing with `matplotlib`
#### 1D Histogram

In [ ]:
plt.hist(table_data['energy'], 1000, log=True)
plt.semilogy()
plt.xlabel("eV")
plt.ylabel("counts per bin")
plt.show()

#### 2D Histogram

In [ ]:
table_image, xedges, yedges, _ = plt.hist2d(table_data['x'], table_data['y'], (1000, 1000), norm=LogNorm())
plt.colorbar()

### Working with large files

The *open()* function supports a *memmap=True* argument that allows the array data of each HDU to be accessed with mmap, rather than being read into memory all at once. This is particularly useful for working with very large arrays that cannot fit entirely into physical memory.

<div class="alert alert-block alert-info">
<h3>Exercise:</h3>
<br>
Construct programmatically a FITS file with 1 image extension 
and save it to disk. Use a numpy array with random numbers as 
data.

Hints: 
- Use `fits.HDUList()` to create a FITS file.
- Use `fits.PrimaryHDU()` for the primary HDU and `fits.ImageHDU()` for the image HDU. 

To look at the documentation of a python object, type one of the following into a python or IPython terminal:

    >>> help(fits.ImageHDU)
    >>> fits.ImageHDU?

</div>

---
# Working with ASCII files

What if your data is in a text file? We've included `throughput.txt` as an example.

In [ ]:
! cat throughput.txt

Python has a built-in method for opening text files like ASCII files, but it's not too friendly. Let's see how it works:

In [ ]:
with open('throughput.txt') as f:
    throughput_data = f.read()
throughput_data

Luckily, Astropy includes a package to handle ASCII files that immediately separates data into rows and columns, and loads it into an easy-to-use table:

In [ ]:
from astropy.io import ascii

Since our file has a couple lines of comments at the top, we need to tell the reader to start reading the header at line 2 and the data at line 3.

In [ ]:
throughput_data = ascii.read('throughput.txt', data_start=3, header_start=2)
throughput_data

We can also redefine the column names, if we want:

In [ ]:
throughput_data = ascii.read('throughput.txt', data_start=3, header_start=2, 
                             names=['lambda', 'throughput'])
throughput_data

In [ ]:
print("Column names: \n", throughput_data.colnames)
print("\nRow 1: \n", throughput_data[1])
print('\nColumn "lambda": \n', throughput_data.field("lambda"))
print("\nNumber of rows: \n", len(throughput_data))

Astropy's ASCII module can read files with a variety of formats and extensions, including comma-separated (CSV), tab-delimited, fixed-width, HTML, reStructuredText, and more. See the [Astropy docs](http://docs.astropy.org/en/stable/io/ascii/#supported-formats) for a complete list.

### Visualizing with `matplotlib`

In [ ]:
plt.plot(throughput_data['lambda'], throughput_data['throughput'])
plt.xlabel('Wavelength ($\mu$m)')
plt.ylabel('Throughput')